## Generate run_files for the stack of offsets

### 1. SLC Stack: generate a stack of coregistered SLC using ISCE/topStack SLC workflow

#### ChileSenAT149

```bash
cd ~/nobak/ChileSenAT149
mkdir DEM; cd DEM
dem.py -a stitch -b -24 -19 -71 -66 -r -s 1 -c

# slc stack
stackSentinel.py -s ./SLC/ -d ./DEM/demLat_S24_S19_Lon_W071_W066.dem.wgs84 -b '-21.8 -20.6 -69.6 -67.0' -a /home/zyunjun/bak/aux/aux_cal/ -o /home/zyunjun/bak/aux/aux_poeorb/ -n '1 2 3' -z 1 -r 1 -C geometry -W slc -useGPU

# interferogram stack
stackSentinel.py -s ./SLC/ -d ./DEM/demLat_S24_S19_Lon_W071_W066.dem.wgs84 -b '-21.8 -20.6 -69.6 -67.0' -a /home/zyunjun/bak/aux/aux_cal/ -o /home/zyunjun/bak/aux/aux_poeorb/ -n '1 2 3' -z 10 -r 30 -c 3 -C geometry -W interferogram -useGPU
```

#### SanAndreasSenDT42
```bash
stackSentinel.py -s ./SLC/ -d ./DEM/demLat_N34_N38_Lon_W124_W118.dem.wgs84 -b '35.4 36.5 -121.1 -120' -a /home/zyunjun/bak/aux/aux_cal/ -o /home/zyunjun/bak/aux/aux_poeorb/ -n '1 2' -z 1 -r 1 -C geometry -W slc -useGPU
```

In [8]:
%matplotlib inline
import os
import glob
import numpy as np
from mintpy.utils import network as pnet, ptime

work_dir = '/Users/yunjunz/data/ChileSenAT149'
os.chdir(work_dir)
print('Go to directory', work_dir)

proj_dir = '/home/zyunjun/nobak/ChileSenAT149'

def get_dense_offset_cmd(proj_dir, date12, gpu_id=None):
    date1, date2 = date12.split('_')
    m_slc = os.path.join(proj_dir, 'merged/SLC/{d}/{d}.slc.full'.format(d=date1))
    s_slc = os.path.join(proj_dir, 'merged/SLC/{d}/{d}.slc.full'.format(d=date2))
    x_file = os.path.join(proj_dir, 'master/IW1.xml')
    prefix = os.path.join(proj_dir, 'merged/offsets/{d}/offset'.format(d=date12))

    cmd = 'cuDenseOffsets.py -m {m} -s {s} --masterxml {x} --outprefix {p} '.format(
        m=m_slc,
        s=s_slc,
        x=x_file,
        p=prefix)
    cmd += ' --kw 300 --kh 100 --nwac 100 --nwdc 1 '
    cmd += ' --ww 256 --wh 256 --sw 10 --sh 10 --raw-osf 2 --corr-osf 64 --redo '

    if gpu_id is not None:
        cmd += ' --gpuid {} '.format(gpu_id)

    return cmd

Go to directory /Users/yunjunz/data/ChileSenAT149


### 2. offset stack: generate a stack of azimuth/range offset using cuDenseOffsets.py

#### 2.1 Generate `date12.list`

In [2]:
### get date12.list
num_conn = 3
date_list_file = os.path.join(work_dir, 'date.list')
date_list = np.loadtxt(date_list_file, dtype=bytes).astype(str).tolist()
date12_list = pnet.select_pairs_sequential(date_list, num_connection=num_conn, date12_format='YYYYMMDD_YYYYMMDD')

## job info
num_pair = len(date12_list)
print('number of jobs: {}'.format(num_pair))

gpu_ids = [str(i) for i in [2,3,4,5,6,7]]
num_gpu = len(gpu_ids)
print('number of GPUs: {}'.format(num_gpu))
num_job_per_gpu = np.ceil(num_pair / num_gpu).astype(int)
print('number of jobs per GPU: {}'.format(num_job_per_gpu))

number of jobs: 306
number of GPUs: 6
number of jobs per GPU: 51


#### 2.2 Generate run_offset_gpu_*.sh

Then copy the run_files to kamb and run each of them in different screen.

In [5]:
run_files = [os.path.join(work_dir, 'run_files_offset/run_offset_gpu_{}.sh'.format(i)) for i in gpu_ids]
log_files = [os.path.join(proj_dir, 'run_files_offset/run_offset_gpu_{}.log'.format(i)) for i in gpu_ids]
run_dir = os.path.dirname(run_files[0])
if not os.path.isdir(run_dir):
    os.makedirs(run_dir)
    print('create directory: {}'.format(run_dir))

fs = []
for i in range(num_gpu):
    f = open(run_files[i], 'w')
    print('open  file: {} with w mode'.format(run_files[i]))

    cmd = 'touch {}'.format(log_files[i])
    f.write(cmd+'\n')
    #print(cmd)

    fs.append(f)

for i in range(num_pair):
    gpu_ind = int(i / num_job_per_gpu)
    log_file = log_files[gpu_ind]
    
    cmd = get_dense_offset_cmd(proj_dir, date12_list[i], gpu_id=gpu_ids[gpu_ind])
    cmd += ' | tee -a {}'.format(log_file)

    f = fs[gpu_ind]
    f.write(cmd+'\n')
    #print(cmd)

for i in range(num_gpu):
    fs[i].close()
    print('close file:', run_files[i])

for i in range(num_gpu):
    cmd = 'chmod +x {}'.format(run_files[i])
    print(cmd)
    os.system(cmd)

open  file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_2.sh with w mode
open  file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_3.sh with w mode
open  file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_4.sh with w mode
open  file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_5.sh with w mode
open  file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_6.sh with w mode
open  file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_7.sh with w mode
close file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_2.sh
close file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_3.sh
close file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_4.sh
close file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_5.sh
close file: /Users/yunjunz/data/ChileSenAT149/run_files_offset/run_offset_gpu_6.sh
close file: /Us

### 2.3 Mask and filter the offset field

In [9]:
off_dir = '/home/zyunjun/nobak/ChileSenAT149/merged/offsets'     # kamb
off_dir = '/Users/yunjunz/data/ChileSenAT149/merged/offsets'     # laptop

off_files = glob.glob(os.path.join(off_dir,'*/offset.bip'))
num_file = len(off_files)
print('number of offset files: {}'.format(num_file))

!which MaskAndFilter.py

prog_bar = ptime.progressBar(maxValue=num_file)
for i in range(num_file):
    off_file = off_files[i]
    off_dir = os.path.dirname(off_file)
    snr_file = os.path.splitext(off_file)[0] + '_snr.bip'
    cmd = 'MaskAndFilter.py -d {d} -s {s} -t 20 -o {o} '.format(d=off_file, s=snr_file, o=off_dir)
    #print(cmd)
    os.system(cmd)

    date12 = os.path.basename(off_dir)
    prog_bar.update(i+1, suffix=date12)
prog_bar.close()
print('Done.')

number of offset files: 306
/Users/yunjunz/development/python/isce2/contrib/stack/stripmapStack/MaskAndFilter.py
[==================================================] 20150928_20151022  507s /    10s 
Done.


### 3.  prepare `geometry` files for offset stack on kamb

```cfg
match_len, match_wid = 256, 256
search_len, search_wid = 10, 10
margin = 50
step_len, step_wid = 100, 300
```

mli_margin = margin + search_len + match_len / 2 = 188

This leads to `$MINTPY_HOME/mintpy/multilook.py -r 300 -a 100 --margin 188 188 188 188`.

Run the following on kamb in the terminal:

```bash
cd ~/nobak/SenDT42F472/merged/geom_master
createWaterMask.py -d ../../DEM/demLat_N34_N38_Lon_W124_W118.dem.wgs84 -l lat.rdr.full -L lon.rdr.full -o waterMask.rdr.full --fill 0

cd ~/nobak/SenDT42F472/merged/offsets/geom_master
multilook.py ../../geom_master/hgt.rdr.full        -r 300 -a 100 --margin 188 188 188 188 -o hgt.rdr
multilook.py ../../geom_master/lat.rdr.full        -r 300 -a 100 --margin 188 188 188 188 -o lat.rdr
multilook.py ../../geom_master/lon.rdr.full        -r 300 -a 100 --margin 188 188 188 188 -o lon.rdr
multilook.py ../../geom_master/los.rdr.full        -r 300 -a 100 --margin 188 188 188 188 -o los.rdr
multilook.py ../../geom_master/shadowMask.rdr.full -r 300 -a 100 --margin 188 188 188 188 -o shadowMask.rdr
multilook.py ../../geom_master/waterMask.rdr.full  -r 300 -a 100 --margin 188 188 188 188 -o waterMask.rdr
```

### Plots

#### stack - azimuth offset

<p align="left">
  <img src="docs/SenDT128/azimuthOffset_1.png">
  <img src="docs/SenDT128/azimuthOffset_2.png">
</p>

#### stack - range offset

<p align="left">
  <img src="docs/SenDT128/rangeOffset_1.png">
  <img src="docs/SenDT128/rangeOffset_2.png">
</p>

#### stack - offset SNR

<p align="left">
  <img src="docs/SenDT128/offsetSNR_1.png">
  <img src="docs/SenDT128/offsetSNR_2.png">
</p>

#### stack - offset SNR - median in form of network

<p align="left">
  <img src="docs/SenDT128/Network.pdf">
  <img src="docs/SenDT128/CoherenceMatrix.pdf">
  <img src="docs/SenDT128/CoherenceHistory.pdf">
</p>
